# Data Analysis of insulin process production

## Goal

Potential projects: Raman spectrometer soft sensor development, Faulty batches detection (before too late), 

## About the data:

The data was downloaded from [Kaggle](https://www.kaggle.com/datasets/stephengoldie/big-databiopharmaceutical-manufacturing/data). It consists of data for 100 batches of insulin, generated by [IndPenSim](http://www.industrialpenicillinsimulation.com/)

![Variables and Parameters](IndPenSim_input_outputs_V2.png) 


## References:

Goldrick S., Stefan, A., Lovett D., Montague G., Lennox B. (2015) The development of an industrial-scale fed-batch fermentation simulation Journal of Biotechnology, 193:70-82.

Goldrick S., Duran-Villalobos C., K. Jankauskas, Lovett D., Farid S. S, Lennox B., (2019) Modern day control challenges for industrial-scale fermentation processes. Computers and Chemical Engineering.

## 1 - Visualisation and cleaning

In [ ]:
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import colormaps
from IPython.display import clear_output, display
from ipywidgets import Output
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import LeaveOneOut
import scipy.stats as stats

from helper import plot_batch_data

data = pd.read_csv('data/100_Batches_IndPenSim_V3.csv', usecols=range(39)) 
data_summary = pd.read_csv('data/100_Batches_IndPenSim_Statistics.csv')

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
# Rewrites Batch ID's correctly
data = data.rename(columns={'2-PAT control(PAT_ref:PAT ref)': 'Batch reference', # Mistaken attribution
                            'Batch reference(Batch_ref:Batch ref)':'Faulty batch', # Mistaken attribution + seems to indicate if a batch is faulty
                            })

# Drop superfluous/unusable columns
data.drop([' 1-Raman spec recorded'], axis=1, inplace=True) # I don't see the point of this column, seems to be a duplicate of Batch reference
data.drop(['Batch ID'], axis=1, inplace=True) # Mysterious column; is not actually a batch ID, closely follows penicilin concentration
data.drop(['Fault flag'], axis=1, inplace=True) # Mysterious column; is not actually a binary flag, closely follows penicilin concentration

# Changes the ID of the columns for more comprehensive names
data = data.rename(columns={'0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref)':'Operator controlled',
                            'Fault reference(Fault_ref:Fault ref)':'Faulty measure'})

# Correct data readability for a categorical (binary) variable:
data = data.rename(columns={'1- No Raman spec': 'Raman spec recorded'})
data['Raman spec recorded'] = data['Raman spec recorded'] - 1

In [ ]:
# Check for missing data:
missing_data = data.isnull().sum()
print(missing_data)

In [ ]:
# Only keep data from the campaign 4 (same as in the paper): Fixed duration, recipe driven
data = data[data['Operator controlled'] == 0]
data = data[data['Raman spec recorded'] == 0] # Raman spec controlled batches are also marked as "recipe driven"
grouped = data.groupby('Batch reference')
filtered_groups = [group for name, group in grouped if len(group) == 1150] # Only keep 230h long batches

data = pd.concat(filtered_groups)
del grouped, filtered_groups

# Drop columns with discrete/categorical variables:
data.drop(['Faulty measure',
           'Operator controlled',
           'Raman spec recorded'],
            axis=1, inplace=True)

# Drop columns with delayed measurements (offline) and values provided by the Raman spectrometer:
data.drop(['PAA concentration offline(PAA_offline:PAA (g L^{-1}))',
           'Offline Penicillin concentration(P_offline:P(g L^{-1}))',
           'Offline Biomass concentratio(X_offline:X(g L^{-1}))',
           'Viscosity(Viscosity_offline:centPoise)',
           'NH_3 concentration off-line(NH3_offline:NH3 (g L^{-1}))',
           'Substrate concentration(S:g/L)',
           'Penicillin concentration(P:g/L)'], axis=1, inplace=True)

# Drop columns with constant values:
data.drop(['Agitator RPM(RPM:RPM)', 'Ammonia shots(NH3_shots:kgs)'], axis=1, inplace=True)

# Creates a copy of the data so that the plot in the following cell can stay interactive
copy_data = data.copy(deep=True)

In [ ]:
plot_batch_data(copy_data)

Important observation: Most batches have a duration of 230h, but some last for slightly longer/shorter times.

- Agitation: Always 100RPM, not useful
- Water for injection/dilution: Same trajectory for all batches
- Air head pressure: Same trajectory for all batches
- Dumped broth flow: Same trajectory for all batches
- Ammonia shots: Same trajectory for all batches
- PAA concentration offline does not show any data on the plot => because taken every 4 hours
- NH3 concentration offline does not show any data on the plot => because taken every 4 hours
- Penicilin concentration offline does not show any data on the plot => because taken every 4 hours
- Biomass concentration offline does not show any data on the plot => because taken every 4 hours
- Viscosity offline does not show any data on the plot => because taken every 4 hours
- What is Batch ID? Seems to be a numerical value related to peniciline concentration
- What is Fault flag? Seems to be a numerical value related to peniciline concentration

//

- Vessel weight and vessel volume should be highly correlated
- Generated heat, oxygen uptake rate and CO2% in outlet should be highly correlated


Some features in the data are there because the data comes from a bioprocess simulation; in reality, these features cannot be measured properly. These features are: 
- On-line Penicillin concentration (We assume we do not have a Raman spectrometer in the tank)

## 2 - Multiway PCA

We are going to apply PCA on the data, to visualize the data and see if we can detect the "good" and the "bad" batches before they end. To do this, we are going to unfold the data, so that we have a row for each batch and a column for every parameter at every time point. It means that every parameter at every timepoint are vectors orthogonal to each other.

In [ ]:
data = data.sort_values(by=['Batch reference', 'Time (h)'])
filtered_operator_driven_data = data # We save this for the later Fourier analysis

# Pivot the DataFrame
variable_names = list(set(data.columns.to_list()) - set('Time (h)'))
data = data.pivot_table(index='Batch reference', columns='Time (h)', values=variable_names)

# Flatten the MultiIndex columns:
data.columns = [f'{var}_t{time}' for var, time in data.columns]

# We keep only one "Faulty batch" column:
columns_to_drop = data.filter(regex="^Faulty batch").columns
data = data.drop(columns_to_drop[1:], axis=1)
data = data.rename(columns={"Faulty batch_t0.2": "Faulty batch"})
del columns_to_drop

data

In [ ]:
# Removes columns with standard deviation of 0, not useful for prediction
# We are able to drop around 40% of the columns!
data = data.loc[:, data.std() != 0]

# Normalization
columns_to_normalize = data.columns.difference(['Faulty batch'])
data[columns_to_normalize] = (data[columns_to_normalize] - data[columns_to_normalize].mean()) / data[columns_to_normalize].std()

NOC_batches = data[data['Faulty batch'] == 0].drop("Faulty batch", axis=1)
faulty_batches = data[data['Faulty batch'] == 1].drop("Faulty batch", axis=1)
del data

# PCA
pca = PCA()
pca.fit(NOC_batches) # Fit with only NOC batches

explained_variance_ratio = pca.explained_variance_ratio_
print("Cumulative explained variance:", explained_variance_ratio.cumsum())

We can see that around 80% of the variance can be explained by 7 components. We are going to use the first 7 PC to describe the data.

In [ ]:
nb_PC = 7
PCA_statistics = pd.DataFrame(columns=['Batch reference', 'Q stat', 'Q threshold', 'T2 stat', 'T2 threshold', 'Faulty batch'])
loo=LeaveOneOut()

for i, (train_index, test_index) in enumerate(loo.split(NOC_batches)):
    # Use .iloc to index by position
    train_data = NOC_batches.iloc[train_index]
    test_data = NOC_batches.iloc[test_index]
    
    # Initialize and fit the PCA model to retreive 
    # all the eigenvalues of the covariance matrix
    pca = PCA()
    pca.fit(train_data)
    #eigenvalues_corr_matrix = pca.explained_variance_ratio_
    eigenvalues_cov_matrix = pca.explained_variance_
    eigenvalues_retained_PC = eigenvalues_cov_matrix[:nb_PC]
    eigenvalues_discarded_PC = eigenvalues_cov_matrix[nb_PC:]
    
    # re_fit the PCA (ugly, but only way to have all eigenvalues 
    # AND projecting on a subset of PC)
    pca = PCA(n_components=nb_PC)
    pca.fit(train_data)
    
    # Transform the data
    test_data_transformed = pca.transform(test_data)
    train_data_transformed = pca.transform(train_data)
    faulty_batches_transformed = pca.transform(faulty_batches)
    
    # Reconstruct the data from the PCA model
    test_data_reconstructed = pca.inverse_transform(test_data_transformed)
    train_data_reconstructed = pca.inverse_transform(train_data_transformed)
    faulty_batches_reconstructed = pca.inverse_transform(faulty_batches_transformed)
    
    alpha = 0.05
    
    # Compute the Q statistic (sum of squared residuals)
    residuals_test_data = test_data - test_data_reconstructed
    residuals_faulty_batches = faulty_batches - faulty_batches_reconstructed
    Q_statistic_test_data = float(np.sum(residuals_test_data ** 2, axis=1).iloc[0])
    
    Q_statistics_faulty_batches = []
    for index, f_batch_df in residuals_faulty_batches.iterrows():
        f_batch = f_batch_df.to_numpy()
        Q_statistic_faulty_batch = np.sum(f_batch ** 2)
        Q_statistics_faulty_batches.append(Q_statistic_faulty_batch)
    
    # Compute the 95% confidence treshold for the Q statistic:
    residuals_train_data = train_data - train_data_reconstructed
    theta_1 = np.sum(eigenvalues_discarded_PC)
    theta_2 = np.sum(eigenvalues_discarded_PC**2)
    g_SPE = theta_2/theta_1
    h_SPE = (theta_1**2)/theta_2
    Q_threshold = g_SPE*stats.chi2.ppf(1 - alpha, h_SPE)
    
    # Compute the Hostelling T² statistic (Mahalanobis distance between origin and scores)
    T2_statistic_test_data = np.sum((test_data_transformed[0] ** 2) / eigenvalues_retained_PC)
    
    T2_statistics_faulty_batches = []
    for f_batch in faulty_batches_transformed:
        T2_statistic_faulty_batch = np.sum((f_batch ** 2) / eigenvalues_retained_PC)
        T2_statistics_faulty_batches.append(T2_statistic_faulty_batch)

    # Compute the 95% confidence treshold for the T² statistic:
    R = nb_PC
    I = len(train_index)
    T2_threshold = R*(I-1)/(I-R) * stats.f.ppf(1-alpha, R, I-R)
    
    # Store the results
    stats_test_row = pd.DataFrame({'Batch reference':[test_data.index[0]], 
                                   'Q stat': [Q_statistic_test_data],
                                   'Q threshold': [Q_threshold],
                                   'T2 stat': [T2_statistic_test_data],
                                   'T2 threshold': [T2_threshold],
                                   'Faulty batch': [0]})
    
    stats_faulty_batches_rows = pd.DataFrame({'Batch reference':faulty_batches.index.to_list(), 
                                   'Q stat': Q_statistics_faulty_batches,
                                   'Q threshold': [Q_threshold]*len(faulty_batches),
                                   'T2 stat': T2_statistics_faulty_batches,
                                   'T2 threshold': [T2_threshold]*len(faulty_batches),
                                   'Faulty batch': [1]*len(faulty_batches)})
    
    PCA_statistics = pd.concat([PCA_statistics, stats_test_row, stats_faulty_batches_rows], ignore_index=True)
    #PCA_statistics = pd.concat([PCA_statistics, stats_faulty_batches_rows], ignore_index=True)

We can now visualize the results. Let's see if batch outcome can be predicted from the whole time series of the process variables : 

In [ ]:
averaged_stats = PCA_statistics.groupby(['Batch reference']).mean()
averaged_stats = averaged_stats.reset_index(drop=True)

# Plot Q statistics
plt.figure(figsize=(12, 6))
for i in range(len(averaged_stats)):
    if averaged_stats['Faulty batch'].iloc[i] == 1.0:
        plt.scatter(averaged_stats.index[i], 
                    averaged_stats['Q stat'].iloc[i], 
                    color='red', 
                    label='Faulty batches' if i == averaged_stats[averaged_stats['Faulty batch']==1].iloc[0].name \
                    else "")
    else:
        plt.scatter(averaged_stats.index[i], 
                    averaged_stats['Q stat'].iloc[i], 
                    color='blue', 
                    label='NOC batches' if i == averaged_stats[averaged_stats['Faulty batch']==0].iloc[0].name \
                    else "")
    
    # Plot thresholds
    plt.hlines(y=averaged_stats['Q threshold'].iloc[i], 
               xmin=averaged_stats.index[i] - 0.5, 
               xmax=averaged_stats.index[i] + 0.5, 
               color='black', 
               linestyles='solid'),
               #label='95% significance threshold')

    
# Calculate the average of the blue and red points
average_blue = averaged_stats[averaged_stats['Faulty batch'] == 0.0]['Q stat'].mean()
average_red = averaged_stats[averaged_stats['Faulty batch'] == 1.0]['Q stat'].mean()

# Add a horizontal blue line at the average height of the blue points
plt.axhline(y=average_blue, color='blue', linestyle='dotted', label='Average of Non-Faulty Batches')
plt.axhline(y=average_red, color='red', linestyle='dotted', label='Average of faulty Batches')
    
# Labeling the plot
plt.xlabel('Batches')
plt.ylabel('Q Statistic')
plt.title('Q Statistics of the PCA (whole batch length)')
plt.xticks(averaged_stats.index)  # Set x-ticks to batch references
plt.legend()

# Hide Y axes label marks
ax = plt.gca()
ax.xaxis.set_tick_params(labelbottom=False)
ax.set_xticks([])

In [ ]:
# Plot T2 statistics:
plt.figure(figsize=(12, 6))
for i in range(len(averaged_stats)):
    if averaged_stats['Faulty batch'].iloc[i] == 1.0:
        plt.scatter(averaged_stats.index[i], 
                    averaged_stats['T2 stat'].iloc[i], 
                    color='red', 
                    label='Faulty batches' if i == averaged_stats[averaged_stats['Faulty batch']==1].iloc[0].name \
                    else "")
    else:
        plt.scatter(averaged_stats.index[i], 
                    averaged_stats['T2 stat'].iloc[i], 
                    color='blue', 
                    label='NOC batches' if i == averaged_stats[averaged_stats['Faulty batch']==0].iloc[0].name \
                    else "")
    
    # Plot thresholds
    plt.hlines(y=averaged_stats['T2 threshold'].iloc[i], 
               xmin=averaged_stats.index[i] - 0.5, 
               xmax=averaged_stats.index[i] + 0.5, 
               color='black', 
               linestyles='solid')
               #label='95% significance threshold')

    
# Calculate the average of the blue and red points
average_blue = averaged_stats[averaged_stats['Faulty batch'] == 0.0]['T2 stat'].mean()
average_red = averaged_stats[averaged_stats['Faulty batch'] == 1.0]['T2 stat'].mean()

# Add a horizontal blue line at the average height of the blue points
plt.axhline(y=average_blue, color='blue', linestyle='dotted', label='Average of Non-Faulty Batches')
plt.axhline(y=average_red, color='red', linestyle='dotted', label='Average of faulty Batches')
    
# Labeling the plot
plt.xlabel('Batches')
plt.ylabel('T2 Statistic')
plt.title('T2 Statistics of the PCA (whole batch length)')
plt.xticks(averaged_stats.index)  # Set x-ticks to batch references
plt.legend()

# Hide Y axes label marks
ax = plt.gca()
ax.xaxis.set_tick_params(labelbottom=False)
ax.set_xticks([])

# Show the plot
plt.show()

We can observe the following two things:

1) The Q statistics are almost all above the 95% confidence threshold and the T2 statistics are all below the confidence threshold regardless of the batch quality. It seems that the tests fail to capture the quality of the batches.

2) The mean statistics (Q and T²) of the NOC batches are slightly lower than the faulty batches. The difference may or may not be significant, we don't have enough data to test it.

In the original paper by Goldrick & al., the authors show that the T² statistic fails to capture the batch quality, as the 95% confidence threshold is never attained by any batch, faulty or not. They highlight that previous research has found the T2 statistic to underestimate faults. \\

Their Q-statistic plot however looks different. All batches are correctly classified as faulty or NOC by the 95% confidence limit, which contrasts to our plot. Goldrick & al constructed the Q-statistic plot by showing the Q-statistic of batches used to create the PCA in the first place. This approach is misleading, as it does not check the prediction accuracy of the test; it's like formulating a hypothesis and validating it with the same data. We tested our PCA with cross-validation, in a way that reflects how newly recorded batches would compare against the PCA model. These results show that the PCA model is likely overfitting the data. This is not really surprising, as there are only 15 NOC batches and thousands of variables. So, what could we do to avoid this overfitting problem?

1) Use less variables: If a fault happens in the process after 200 hours, it is more likely that the first hours of the process are not the cause of this effect. The cause-to-effect time should be limited by a certain value. This means that we can use data only from the last X measures of the process to detect faults, in a moving window manner. It would avoid adding unnecessary variables to the PCA model that most likely contain NOC data anyway.

2) Normalize the data differently: Some variables have variability over time heavily influenced by noise. We can guess that based on our mechanistic understanding of the process. For example, recorded temperature should be pretty uniform, high frequency changes are noise that we do not want to have in our PCA model. To avoid that, we could apply a smoothing function to the data, or use a low-pass filter. The method of choice has to be applicable in real time, since we want real-time monitoring of the process.

## 3 - Fourier Analysis 

In [ ]:
# Compute the DFT for each batch:
grouped_batches = filtered_operator_driven_data.groupby('Batch reference')
fourier_all_operator_batches = []
for name, batch in grouped_batches:
    N = len(batch)
    raw_fourier_batch = np.abs(np.fft.fft(batch.to_numpy(), axis=0))/N # Get the amplitude of the signal
    fourier_operator_batch = pd.DataFrame(raw_fourier_batch, columns=filtered_operator_driven_data.columns)
    fourier_operator_batch['Batch reference'] = batch['Batch reference'].reset_index(drop=True)
    fourier_operator_batch['Faulty batch'] = batch['Faulty batch'].reset_index(drop=True)
    fourier_operator_batch['Frequency (1/h)'] = np.fft.fftfreq(len(batch), d=0.2)
    fourier_all_operator_batches.append(fourier_operator_batch)
    
fourier_all_operator_batches = pd.concat(fourier_all_operator_batches)
fourier_all_operator_batches.drop(['Time (h)'], axis=1, inplace=True)
fourier_all_operator_batches = fourier_all_operator_batches.sort_values(by=['Batch reference', 'Frequency (1/h)'])
fourier_all_operator_batches.reset_index(drop=True, inplace=True)

# Removing the DC components out of the Fourier transforms:
columns_to_modify = fourier_all_operator_batches.columns.difference(['Frequency (1/h)', 'Batch reference', 'Faulty batch'])
fourier_all_operator_batches.loc[fourier_all_operator_batches['Frequency (1/h)'] == 0, columns_to_modify] = 0
fourier_all_operator_batches

In [ ]:
variable_list_2 = fourier_all_operator_batches.columns
variable_plot_selection_2 = widgets.Dropdown(options=variable_list_2, value = 'Temperature(T:K)')
variable_plot_selection_2

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
bp = fourier_all_operator_batches.groupby('Batch reference').plot(x = 'Frequency (1/h)', y = variable_plot_selection_2.value,   ax=ax, legend = False, )
ax.set_title('Summary of Campaigns, Frequency domain (DC removed)')
ax.set_xlabel('Frequency (1/h)')
ax.set_ylabel(variable_plot_selection_2.value)

## 4 - Correlation Analysis

The time series are heavily correlated one with the other. We leverage this fact to reduce dimensionality with PCA. Now, we want to have a closer look at the correlation structure of the process. This will help us find the size of the temporal window we are going to use to construct PCA with less variables (and hopefully, better detection of faulty batches).

Lagged Cross-Correlation Matrix:

For each lag ll, compute a cross-correlation matrix C(l)C(l), which contains correlations between each pair of variables at time tt and t−lt−l. Summarize C(l)C(l) to a single measure, such as the average cross-correlation or maximum cross-correlation across all pairs. This will give you a measure of how much variables are correlated with their past values at each lag ll.

The triangle comes from the fact that I have a BIASED correlation: points at the beginning and end of the series have less data to be evaluated. The triangular shape happens when there is a biased (0-padded, or finite) correlation between signals. To remove this triangular shape, one can simply remove the DC component/ the offset of the signals.

In [ ]:
# Autocorrelation plot:
# I am going to see correlations, but the more shifted they are, the less meaningful they are.
# Maybe this does not help with finding the size of the temporal window?
autocorrelated_all_operator_batches = list()
for name, batch in grouped_batches:
    # Compute the cross-correlation of the series with itself: (Could also use the Fourier transforms from above)
    mean_1 = batch['Oxygen Uptake Rate(OUR:(g min^{-1}))'].mean()
    mean_2 = batch['Oxygen in percent in off-gas(O2:O2  (%))'].mean()
    autocorrelation = np.convolve(batch['Oxygen Uptake Rate(OUR:(g min^{-1}))'] - mean_1, 
                                  batch['Oxygen in percent in off-gas(O2:O2  (%))'].iloc[::-1] - mean_2)
    # Normalize the result
    autocorrelation = autocorrelation / (np.linalg.norm(batch['Oxygen Uptake Rate(OUR:(g min^{-1}))']) \
                                         * np.linalg.norm(batch['Oxygen in percent in off-gas(O2:O2  (%))']))
    autocorr_operator_batch = pd.Series(autocorrelation)
    autocorrelated_all_operator_batches.append(autocorr_operator_batch)

autocorrelated_all_operator_batches = pd.concat(autocorrelated_all_operator_batches, axis=1)
n = len(autocorrelated_all_operator_batches)
autocorrelated_all_operator_batches['time shift'] = np.arange(-n // 2, n // 2)

In [ ]:
variable_list_3 = autocorrelated_all_operator_batches.columns
variable_plot_selection_3 = widgets.Dropdown(options=variable_list_3)
variable_plot_selection_3

In [ ]:
autocorrelated_all_operator_batches.plot(x='time shift', y=variable_plot_selection_3.value)
ax.set_xlabel('cross-autocorrelation')
ax.set_ylabel(variable_plot_selection_3.value)

plt.show()